# <p align="center">**5.2 Tiamat: Implementação do Algoritmo Genético** 🧬</p>
 
##### <p align="center">**Autores:**  Glauber Nascimento, Júlia Guedes & Lorena Ribeiro</p>
##### <p align="center">**Orientador:**  Daniel Roberto Cassar </p>

<div style="background-color: lightblue; font-size: 18px; padding: 10px;">
<div style="text-align: justify"><strong>Objetivo:</strong> Implementação do Algorítmo Genético </div>

## 🪨 **Introdução**

<p align="justify">
A dureza, é um valor quantitativo de resistência a deformações em um material. A partir desse conceito, o minerologista alemão Friedrich Mohs criou a escala de Mohs, que é uma escala da dureza de minerais com o objetivo de identifica-los em campo, testando os minerais contra alguns objetos, como a unha, uma moeda ou um prego. Cada resistência do material ao objeto representa um valor dentro da escala de Mohs, contendo valores de 1 a 10, quanto maior, maior a dureza do mineral.
</p>

<p align="justify">
O dataset usado no trabalho foi retirado dos estudos esperimentais reportados no "Physical and Optical Properties of Minerals CRC Handbook of Chemistry and Physics" e "The American Mineralogist Crystal Structure Database." Os atributos presentes no dataset são: Número de eletrons, Número de eletrons na camada de valência, Eletronegatividade de Pauling no estado de oxidação mais comum, Raios atomicos covalentes, Raio de Van der Walls e energia de ionização do neutro
</p>

<p align="justify">
Nesse trabalho, queremos encontrar o candidato que possua as propriedades que maximizem a dureza, visto que o material com maior dureza são mais resistentes ao desgaste e mais duráveis. Assim, entramos em um problema de <b>otimização de maximização</b>, que consiste em encontrar um ponto ótimo que maximiza a função objetivo. No futuro, será possivel encontrar a fórmula química do material sintetizado a partir das propriedades otimizadas pelo nosso algoritmo genético, visando a maximização da dureza.
</p>

<p align="justify">

</p>

<p align="justify">
Para a encontrar o candidato que buscamos, iremos usar os Algoritmos Genéticos para identifica-los, com uma função objetivo que possua um modelo de Aprendizado de Maquina, inicialmente testada com k-NN e Árvore de Decisão, para prever a dureza e, por fim, encontrar-mos o minério com a maior dureza.
</p>

## 📚**Importação de bibliotecas & Dataset**

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [16]:
df = pd.read_csv("Mineral_Dataset_Supplementary_Info.csv").dropna().drop(columns=["Unnamed: 0"])
display(df)

,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,2.3,110.0,23.000000,36.666667,2.666667,82.598467,8.504133,2.146667,2.006667,1.253333,0.456803,7.666667
1,5.5,406.0,30.472136,9.902439,4.682927,19.813180,11.456151,2.700244,1.676829,0.868293,0.522909,0.743223
2,5.5,406.0,30.472464,10.410256,4.923077,20.931371,11.541405,2.753590,1.703846,0.894359,0.497498,0.781345
3,5.5,476.0,61.142136,11.609756,4.682927,23.659644,11.487395,2.763659,1.714634,0.848780,0.519474,1.491272
4,5.5,476.0,61.142464,12.205128,4.923077,24.975089,11.574251,2.820256,1.743590,0.873846,0.493887,1.567755
...,...,...,...,...,...,...,...,...,...,...,...,...
617,3.8,46.0,9.133000,23.000000,4.000000,48.719500,9.877100,2.115000,1.905000,1.120000,0.478880,4.566500
618,4.5,86.0,6.674328,14.333333,5.166667,30.645954,11.862733,2.861667,1.700000,0.901667,0.487172,1.112388
619,4.0,38.0,7.134332,19.000000,4.000000,40.689515,11.506150,2.545000,1.765000,0.920000,0.479405,3.567166
620,7.5,86.0,8.841328,14.333333,5.000000,30.550687,11.543000,2.831667,1.735000,0.890000,0.489507,1.473555


## **Colunas do dataset** 📋

| Features    | Descrição                                                                                      |
|-----------|-----------------------------------------------------------------------------------------------|
| `Hardness`   | Dureza                                                |
| `allelectrons_Total`  | Números de eletrons totais                                                     |
| `density_Total`  | Densidade total               |
| `allelectrons_Average`  | Média do número de eletrons              |
| `val_e_Average`  | Eletrons na camada de valência              |
| `atomicweight_Average`  | Peso atômico médio               |
| `ionenergy_Average`  | Eletronegatividade de Pauling no estado de oxidação mais comum              |
| `el_neg_chi_Average`  | Média da quantidade de pontos de concavidade nos contornos               |
| `R_vdw_element_Average`  | Raio de Van der Walls               |
| `R_cov_element_Average`  | Raios atomicos covalentes               |
| `zaratio_Average`  | Raio Atômico médio               |
| `density_Average`  | Densidade média               |


In [17]:
semente_aleatoria = 71012
tamanho_teste = 0.1

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=tamanho_teste, random_state=semente_aleatoria, shuffle=True
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

In [18]:
X_treino = df_treino.drop(columns=["Hardness"]).values
X_teste = df_teste.drop(columns=["Hardness"]).values

y_treino = df_treino["Hardness"].values.ravel()
y_teste = df_teste["Hardness"].values.ravel()

In [19]:
import pickle
import pprint

from funcoes_tiamat import cria_populacao_feature as cria_populacao
from funcoes_tiamat import funcao_objetivo_pop_feature as funcao_objetivo_ag
from funcoes_tiamat import selecao_torneio_max as funcao_selecao
from funcoes_tiamat import cruzamento_ponto_simples as funcao_cruzamento
from funcoes_tiamat import mutacao_simples as funcao_mutacao

cria população - random.int

função objetivo - com kNN ou Árvore de Decisão 

seleção - Torneio

cruzamento - Ponto Simples

mutação - random respeitando o range de cada índice do indivíduo

In [20]:
with open("dicionario_intervalos.pkl", "rb") as dicionario_range:
    dicionario_range = pickle.load(dicionario_range)

In [27]:
dicionario_range

{'allelectrons_Total': [6, 15300.0],
 'density_Total': [1, 645.0],
 'allelectrons_Average': [4, 70.0],
 'val_e_Average': [2, 10.0],
 'atomicweight_Average': [8, 170.0],
 'ionenergy_Average': [8, 15.0],
 'el_neg_chi_Average': [2, 5.0],
 'R_vdw_element_Average': [1, 5.0],
 'R_cov_element_Average': [1, 5.0],
 'zaratio_Average': [0, 5.0],
 'density_Average': [0, 15.0]}

In [21]:
dicionario_range.pop("Hardness")


[1, 10.0]

In [22]:
INTERVALOS = [i for i in dicionario_range.values()]

In [23]:
TAMANHO_POPULACAO = 10000
NUM_GERACOES = 50
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.05
TAMANHO_TORNEIO = 5

In [24]:
populacao = cria_populacao(TAMANHO_POPULACAO, INTERVALOS)
#pprint(populacao)
print(populacao)

[[11436, 195, 28, 2, 32, 13, 3, 3, 3, 3, 11], [1588, 342, 35, 8, 23, 13, 2, 2, 3, 1, 0], [5762, 313, 26, 5, 55, 14, 4, 1, 4, 3, 10], [5245, 367, 10, 3, 147, 12, 4, 2, 2, 0, 11], [7636, 590, 49, 2, 120, 13, 4, 1, 1, 2, 5], [11640, 357, 29, 2, 90, 12, 4, 3, 3, 4, 5], [3634, 295, 29, 4, 49, 14, 3, 3, 2, 4, 13], [14728, 101, 24, 8, 79, 8, 2, 4, 3, 3, 13], [12873, 303, 51, 5, 93, 9, 3, 4, 2, 3, 4], [7948, 499, 25, 6, 139, 13, 4, 3, 4, 4, 5], [6258, 521, 68, 4, 124, 8, 2, 1, 4, 3, 9], [4127, 139, 20, 6, 75, 10, 3, 1, 3, 2, 10], [5625, 463, 47, 7, 18, 10, 2, 4, 4, 1, 10], [7359, 153, 67, 5, 102, 8, 4, 1, 3, 3, 6], [8709, 634, 65, 4, 24, 11, 4, 4, 4, 3, 8], [6131, 38, 9, 5, 85, 10, 2, 3, 2, 0, 6], [3507, 82, 69, 6, 161, 8, 4, 1, 1, 0, 0], [10706, 146, 44, 3, 74, 9, 4, 3, 3, 1, 1], [10487, 441, 66, 7, 54, 11, 3, 4, 1, 4, 13], [4672, 138, 67, 3, 11, 9, 3, 4, 2, 3, 11], [6790, 205, 18, 7, 98, 12, 3, 4, 4, 4, 2], [4920, 390, 51, 6, 19, 10, 3, 3, 1, 2, 2], [2567, 419, 40, 2, 105, 11, 2, 1, 4, 3, 5]

In [25]:
hall_da_fama = []
maiores_fitness = []
paciencia = 0

contador = 0
maior_fitness = 0
 
while maior_fitness < 9 and paciencia <= 10 and contador < 100:
    contador += 1

    # Seleção
    fitness = funcao_objetivo_ag(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)

    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO, INTERVALOS)

    # Nova avaliação
    fitness = funcao_objetivo_ag(proxima_geracao)
    maior_fitness = max(fitness)
    maiores_fitness.append(maior_fitness)

    if len(maiores_fitness) > 1:
        diferenca = abs(maior_fitness - maiores_fitness[-2])
    else:
        diferenca = float('inf')  

    if diferenca < 0.1:
        paciencia += 1
    else:
        paciencia = 0

    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    

    # Atualiza população
    populacao = proxima_geracao

    print(f"Geração {contador}: maior fitness = {maior_fitness}")


Geração 1: maior fitness = [6.16]
Geração 2: maior fitness = [6.26]
Geração 3: maior fitness = [6.42]
Geração 4: maior fitness = [6.86]
Geração 5: maior fitness = [7.46]
Geração 6: maior fitness = [7.46]
Geração 7: maior fitness = [7.82]
Geração 8: maior fitness = [7.82]
Geração 9: maior fitness = [7.82]
Geração 10: maior fitness = [7.82]
Geração 11: maior fitness = [7.82]
Geração 12: maior fitness = [7.82]
Geração 13: maior fitness = [8.18]
Geração 14: maior fitness = [7.82]
Geração 15: maior fitness = [7.82]
Geração 16: maior fitness = [7.82]
Geração 17: maior fitness = [7.82]
Geração 18: maior fitness = [7.82]
Geração 19: maior fitness = [7.82]
Geração 20: maior fitness = [7.82]
Geração 21: maior fitness = [7.82]
Geração 22: maior fitness = [7.82]
Geração 23: maior fitness = [7.82]
Geração 24: maior fitness = [7.82]
Geração 25: maior fitness = [7.82]


In [30]:
maior_fitness = max(hall_da_fama)
melhor_indice = hall_da_fama.index(maior_fitness)
melhor_individuo = hall_da_fama[melhor_indice]

print("Melhor indivíduo:", melhor_individuo)

Melhor indivíduo: [15049.13644368199, 47, 9, 5, 14, 8, 3, 1, 1, 0, 3]


In [28]:
df_artificial = pd.read_csv("Artificial_Crystals_Dataset 1.csv")
df_filtrado = df_artificial[df_artificial["Hardness (Mohs)"] == 9]

df_filtrado

,Unnamed: 0,Formula,Crystal structure,Hardness (Mohs),allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
36,36,SrO(B2O3)2,orthorhombic,9.0,114.0,12.029324,9.5,4.666667,20.236434,11.184467,2.765833,1.734167,0.811667,0.481969,1.002444
